## Assignment 2 Data Analysis using Pandas

This assignment will contain 1 question with details as below. The due date is October 17 (Sunday), 2021 23:59PM. Each late day will result in 20% loss of total points.

### Question 1 (100 points) How to create the next Squid Game?

![hitsong](https://pbs.twimg.com/media/E_U24rOVEAAfrJm?format=jpg)

The 21th century has witnessed the technological advancement in music industry that allowed consumers to store music in hard disks such as MP3 or iPods. The increasing prevalence of smart phones and the digitization of music prompted the establishment and wide usage of numerous music-listening apps such as Spotify, Google Play Music and Apple Music, among others, that gradually replaced CDs. Such switch of music consumptions, from purchasing physical albums to purchasing the single track, not only changed the customer experience, but also fundamentally changed the economics of the music industry. 

Due to such a music industry evolution, Chris Anderson (2004) proposed the long tail theory to characterize the music consumption in digital era, in which a large portion of tracks that were once unknown have gained certain level of popularity altogether to form a long tail of the consumption distribution. This implies that the popularity of the music and artists may spread within a larger range, increasing sales of less known tracks from nearly zero to few.

More recently, the emergence of streaming platform designs such as Pandora and Spotify, as well as the utilization of Artificial Intelligence into music recommendations have gradually exhibited a spill-over effect (Aguiar and Waldfogel 2018) – music listened by other users with similar histories are recommended, thus increasing the music popularity as it spreads from several users to a larger group. This pushed a short list of tracks to become uniquely popular. In 2018, Professor Serguei Netessine from Wharton University of Pennsylvania stated in his podcast that, “We found that, if anything, you see more and more concentration of demand at the top”. Although the podcast focused on movie sales, experiences goods like theater and music sales occur in a similar fashion. In the book “All you need to know about the music industry” by Passman (2019), he highlighted key differences between music business in the streaming era and record sales. In the days of record sales, artists get paid the same money for each record sold, regardless of whether a buyer listened to it once or a thousand times.  But today, the more listens the music tracks have, the more money the artists make. Meanwhile, records sales do not have strong spillover effects as fans of different artists/genres will purchase what they like anyway. In fact, a hit album would bring a lot of people into record stores, and that increased the chances of selling other records. But in the streaming world, that’s no longer true. The more listens one artist gets, the less money other artists would make. In other words, the music consumption is undertaking a radical shift which may affect the definition of popularity in the streaming era, however, it is yet severely underexplored.

Inspired by the evolution of music industry in the recent decades and the recent debunk of long tail theory given a high concentration of popularity for a short list of tracks, this assignment aims to investigate the popularity of music tracks on streaming platform, largely different and not extensively explored about compared to that measured by album sales. In particular, rather than considering the level of advertisement, the inclusion in playlists of Spotify 100 as Luis Aguiar and Joel Waldfogel (2018) have noted. 

References:
- Aguiar, L. & Joel Waldfogel, Platforms, Promotion, and Product Discovery: Evidence from Spotify Playlists; JRC Digital Economy Working Paper 2018-04; JRC Technical Reports, JRC112023
- Passman (2019), All You Need to Know About the Music Business: 10th Edition, Simon & Schuster, US



**Question 1.1** (30 points): We will retrieve the  information from the top 100 songs on [Spotifycharts](https://spotifycharts.com/) on September 30th-October 4th. For each day on the list, we can scrape the following characteristics from the information page. For example, from the ["Global Top 200 on September 30"](https://spotifycharts.com/regional/global/daily/2021-09-30), we want to extract the information about the top song **STAY** as:
- spotify id (5PjdY0CKGZdEuoNab3yDmX)
- Song name (STAY (with Justin Bieber))
- Artist (The Kid LAROI)
- Number of streams (7,714,466)

![spotifycharts](https://aristake.com/wp-content/uploads/2021/09/Spotify-charts-HEADER-1.png)


After scraping the top 100 songs, save the data as a dataframe ```spotify_top_songs_global```. 

Then similarly, let's try to scrape information from the top 100 songs of Portugal market and Japanese market on Septebmer 30th-October 4th, respectively. save the data as dataframes ```spotify_top_songs_portugal``` and ```spotify_top_songs_japan```.


You can concatenate these three dataframes as ```spotify_top_songs``` for next question. 

Note: if you are not able to scrape the data, download the csv files from the top right corner of the website, but you will not receive the scores from this question.

Hint: you can play with the website to check the correct url for each chart.

In [1]:
import pandas as pd
import numpy as np
import requests #send HTTP requests, get info from webpages
from bs4 import BeautifulSoup #extract data from HTML files
from time import sleep #don't overwhelm servers, pause in the code
import cloudscraper
from datetime import date, timedelta, datetime #date->check date, timedelta -> get data for each day in the observed period
import json

Given that Spotify enforces anti-bot measures to prevent web scraping, we will rely on a package namede ```cloudcraper``` to bypass the mechanism. Essentially you could use the following code to scrape such website easily:

In [2]:
# Question 1.1 - Global

scraper = cloudscraper.create_scraper()  # returns a CloudScraper instance
dates = []
url_list = []
final = []

url = "https://spotifycharts.com/regional/global/daily/"
start_date = date(2021, 9, 30)
end_date = date(2021, 10, 4)
delta = end_date - start_date
    
for i in range(delta.days+1): #delta is short by one day. Add additonal day using .days+1
    day = start_date+timedelta(days=i) #timedelta - duration of time used for manipulating dates
    day_string = day.strftime("%Y-%m-%d") #strftime returns a string representing date and time using date, time or datetime object
    dates.append(day_string) #add each relevant day to our list dates[]

def add_url():
     for date in dates:
        c_string= url+date #add the date to our basic url
        url_list.append(c_string) #add each new, final url to our list of urls
add_url()

def song_scrape_gl(x): #search through the text which we got from Beautiful Soup to find each table row. Then, analyze the HTML to see what elements we need to select
    for j,tr in enumerate(songs.find("tbody").findAll("tr")):
        artist= tr.find("td", {"class": "chart-table-track"}).find("span").text
        artist= artist.replace("by ","").strip()    
        title= tr.find("td",{"class": "chart-table-track"}).find("strong").text
        songid= tr.find("td", {"class": "chart-table-image"}).find("a").get("href")
        songid= songid.split("track/")[1]
        url_date= x.split("daily/")[1]
        streams= tr.find("td", {"class": "chart-table-streams"}).text.replace(",","")
        rank= tr.find("td",{"class": "chart-table-position"}).text
        final.append([title, artist, songid, streams, url_date,rank])
        if j == 99:
            break

for u in url_list: #find the information we want to disect for each url by using the function we defined before    
    read_pg= scraper.get(u)  
    sleep(2)    
    soup = BeautifulSoup(read_pg.text, "html.parser")    
    songs = soup.find("table", {"class":"chart-table"}) 
    song_scrape_gl(u)

spotify_top_songs_global = pd.DataFrame(final, columns= ["Title", "Artist", "Song ID", "Streams", "Chart Date","Rank"]) #save the data in a dataframe
spotify_top_songs_global["Rank"] = spotify_top_songs_global["Rank"].astype(int)
spotify_top_songs_global["Country"] = "Global"
spotify_top_songs_global

,Title,Artist,Song ID,Streams,Chart Date,Rank,Country
0,STAY (with Justin Bieber),The Kid LAROI,5PjdY0CKGZdEuoNab3yDmX,7714466,2021-09-30,1,Global
1,INDUSTRY BABY (feat. Jack Harlow),Lil Nas X,5Z9KJZvQzH6PFmb8SNkxuk,6517968,2021-09-30,2,Global
2,Heat Waves,Glass Animals,02MWAaffLxlfxAUY7c5dvx,4460880,2021-09-30,3,Global
3,My Universe,"Coldplay, BTS",3FeVmId7tL5YN8B7R3imoM,4142687,2021-09-30,4,Global
4,Bad Habits,Ed Sheeran,6PQ88X9TkUIAUIZJHW2upE,4077321,2021-09-30,5,Global
...,...,...,...,...,...,...,...
495,Tiroteo - Remix,"Marc Seguí, Rauw Alejandro, Pol Granch",4OwhwvKESFtuu06dTgct7i,972729,2021-10-04,96,Global
496,Save Your Tears,The Weeknd,5QO79kh1waicV47BqGRL3g,968693,2021-10-04,97,Global
497,Before You Go,Lewis Capaldi,2gMXnyrvIjhVBUZwvLZDMP,961416,2021-10-04,98,Global
498,Chosen (feat. Ty Dolla $ign),"Blxst, Tyga",1dIWPXMX4kRHj6Dt2DStUQ,954619,2021-10-04,99,Global


In [3]:
# Question 1.1 - Portugal

scraper = cloudscraper.create_scraper()
dates=[]
url_list=[]
final = []

url = "https://spotifycharts.com/regional/pt/daily/"
start_date= date(2021, 9, 30)
end_date= date(2021, 10, 4) 
delta= end_date-start_date 
for i in range(delta.days+1):  
    day = start_date+timedelta(days=i) 
    day_string = day.strftime("%Y-%m-%d")
    dates.append(day_string)

def add_url():
     for date in dates:
        c_string= url+date
        url_list.append(c_string)
add_url()

def song_scrape(x): 
    for j,tr in enumerate(songs.find("tbody").findAll("tr")):
        artist= tr.find("td", {"class": "chart-table-track"}).find("span").text
        artist= artist.replace("by ","").strip()    
        title= tr.find("td",{"class": "chart-table-track"}).find("strong").text
        songid= tr.find("td", {"class": "chart-table-image"}).find("a").get("href")
        songid= songid.split("track/")[1]
        url_date= x.split("daily/")[1]
        streams= tr.find("td", {"class": "chart-table-streams"}).text.replace(",","")
        rank= tr.find("td",{"class": "chart-table-position"}).text
        final.append([title, artist, songid, streams, url_date,rank])
        if j == 99:
            break

for u in url_list:    
    read_pg= scraper.get(u)  
    sleep(2)    
    soup = BeautifulSoup(read_pg.text, "html.parser")    
    songs = soup.find("table", {"class":"chart-table"}) 
    song_scrape(u)

spotify_top_songs_pt = pd.DataFrame(final, columns= ["Title", "Artist", "Song ID", "Streams", "Chart Date","Rank"]) 
spotify_top_songs_pt["Rank"] = spotify_top_songs_pt["Rank"].astype(int)
spotify_top_songs_pt["Country"] = "Portugal"
spotify_top_songs_pt

,Title,Artist,Song ID,Streams,Chart Date,Rank,Country
0,INDUSTRY BABY (feat. Jack Harlow),Lil Nas X,5Z9KJZvQzH6PFmb8SNkxuk,49560,2021-09-30,1,Portugal
1,Pepas,Farruko,5fwSHlTEWpluwOM0Sxnh5k,45858,2021-09-30,2,Portugal
2,love nwantiti (ah ah ah),CKay,2Xr1dTzJee307rmrkt8c0g,39748,2021-09-30,3,Portugal
3,Quer Voar,Matuê,7aZusA4cWXz3Wv9e9uhavz,38730,2021-09-30,4,Portugal
4,STAY (with Justin Bieber),The Kid LAROI,5PjdY0CKGZdEuoNab3yDmX,34311,2021-09-30,5,Portugal
...,...,...,...,...,...,...,...
495,Bored,Billie Eilish,04sN26COy28wTXYj3dMoiZ,7122,2021-10-04,96,Portugal
496,"Trava na Pose, Chama no Zoom, Dá um Close (fea...","DJ Patrick Muniz, Dj Olliver, Mc Topre",0DsPj89zlY3Us7xb5cXK5h,7046,2021-10-04,97,Portugal
497,Ramenez la coupe à la maison,Vegedream,1m0UFnuTktOkksvjbF9z0m,7026,2021-10-04,98,Portugal
498,Volta,T-Rex,275Brpw83x3q0mBa9MpCx3,7020,2021-10-04,99,Portugal


In [4]:
# Question 1.1 - Japan
scraper = cloudscraper.create_scraper()
dates=[]
url_list=[]
final = []

url = "https://spotifycharts.com/regional/jp/daily/"
start_date= date(2021, 9, 30)
end_date= date(2021, 10, 4) 
delta= end_date-start_date 
for i in range(delta.days+1):
    day = start_date+timedelta(days=i)
    day_string = day.strftime("%Y-%m-%d")
    dates.append(day_string)

def add_url():
     for date in dates:
        c_string= url+date
        url_list.append(c_string)
add_url()

def song_scrape(x): 
    for j,tr in enumerate(songs.find("tbody").findAll("tr")):
        artist= tr.find("td", {"class": "chart-table-track"}).find("span").text
        artist= artist.replace("by ","").strip()    
        title= tr.find("td",{"class": "chart-table-track"}).find("strong").text
        songid= tr.find("td", {"class": "chart-table-image"}).find("a").get("href")
        songid= songid.split("track/")[1]
        url_date= x.split("daily/")[1]
        streams= tr.find("td", {"class": "chart-table-streams"}).text.replace(",","")
        rank= tr.find("td",{"class": "chart-table-position"}).text
        final.append([title, artist, songid, streams, url_date,rank])
        if j == 99:
            break

for u in url_list:    
    read_pg= scraper.get(u)  
    sleep(2)    
    soup = BeautifulSoup(read_pg.text, "html.parser")    
    songs = soup.find("table", {"class":"chart-table"}) 
    song_scrape(u)

spotify_top_songs_jp = pd.DataFrame(final, columns= ["Title", "Artist", "Song ID", "Streams", "Chart Date","Rank"]) 
spotify_top_songs_jp["Rank"] = spotify_top_songs_jp["Rank"].astype(int)
spotify_top_songs_jp["Country"] = "Japan"
spotify_top_songs_jp


,Title,Artist,Song ID,Streams,Chart Date,Rank,Country
0,Permission to Dance,BTS,5eXBXreN3d1zdj6Sa8dS0u,211629,2021-09-30,1,Japan
1,Butter,BTS,2bgTY4UwhfBYhGT4HUYStN,210937,2021-09-30,2,Japan
2,水平線,back number,5m1i6hq7dmRlp3c1utE48L,206640,2021-09-30,3,Japan
3,ドライフラワー,優里,7dH0dpi751EoguDDg3xx6J,202431,2021-09-30,4,Japan
4,Cry Baby,Official HIGE DANdism,6wDntdm888mDo458RaYjGl,193861,2021-09-30,5,Japan
...,...,...,...,...,...,...,...
495,10月無口な君を忘れる,あたらよ,2YQ8TlTmNheRI3VafoDpod,38331,2021-10-04,96,Japan
496,花束,back number,3QIAwtEEDOrv0g5NKCGrXZ,38136,2021-10-04,97,Japan
497,優しい彗星,YOASOBI,19fhOFi6pNGeZe5uiFlm7c,37380,2021-10-04,98,Japan
498,のびしろ,Creepy Nuts,3bbIIVIwBoLqVcLebiEJFo,37239,2021-10-04,99,Japan


In [5]:
frames = [spotify_top_songs_global, spotify_top_songs_pt, spotify_top_songs_jp] #simply concatenate the data into one big dataframe

spotify_top_songs = pd.concat(frames).reset_index(drop=True)

spotify_top_songs

,Title,Artist,Song ID,Streams,Chart Date,Rank,Country
0,STAY (with Justin Bieber),The Kid LAROI,5PjdY0CKGZdEuoNab3yDmX,7714466,2021-09-30,1,Global
1,INDUSTRY BABY (feat. Jack Harlow),Lil Nas X,5Z9KJZvQzH6PFmb8SNkxuk,6517968,2021-09-30,2,Global
2,Heat Waves,Glass Animals,02MWAaffLxlfxAUY7c5dvx,4460880,2021-09-30,3,Global
3,My Universe,"Coldplay, BTS",3FeVmId7tL5YN8B7R3imoM,4142687,2021-09-30,4,Global
4,Bad Habits,Ed Sheeran,6PQ88X9TkUIAUIZJHW2upE,4077321,2021-09-30,5,Global
...,...,...,...,...,...,...,...
1495,10月無口な君を忘れる,あたらよ,2YQ8TlTmNheRI3VafoDpod,38331,2021-10-04,96,Japan
1496,花束,back number,3QIAwtEEDOrv0g5NKCGrXZ,38136,2021-10-04,97,Japan
1497,優しい彗星,YOASOBI,19fhOFi6pNGeZe5uiFlm7c,37380,2021-10-04,98,Japan
1498,のびしろ,Creepy Nuts,3bbIIVIwBoLqVcLebiEJFo,37239,2021-10-04,99,Japan


In [6]:
 #Just for me out of couriosity ;)
spotify_top_songs_dropped= spotify_top_songs.drop_duplicates(subset=['Song ID']).reset_index(drop=True) #find the unique songs of the concatenated dataframe
spotify_top_songs_dropped 

,Title,Artist,Song ID,Streams,Chart Date,Rank,Country
0,STAY (with Justin Bieber),The Kid LAROI,5PjdY0CKGZdEuoNab3yDmX,7714466,2021-09-30,1,Global
1,INDUSTRY BABY (feat. Jack Harlow),Lil Nas X,5Z9KJZvQzH6PFmb8SNkxuk,6517968,2021-09-30,2,Global
2,Heat Waves,Glass Animals,02MWAaffLxlfxAUY7c5dvx,4460880,2021-09-30,3,Global
3,My Universe,"Coldplay, BTS",3FeVmId7tL5YN8B7R3imoM,4142687,2021-09-30,4,Global
4,Bad Habits,Ed Sheeran,6PQ88X9TkUIAUIZJHW2upE,4077321,2021-09-30,5,Global
...,...,...,...,...,...,...,...
262,Shape of You,Ed Sheeran,7qiZfU4dY1lWllzX7mPBI3,41550,2021-10-02,94,Japan
263,Anniversary,HIRAIDAI,118KEprYqS3XNWdBoKzkEH,42347,2021-10-04,82,Japan
264,10月無口な君を忘れる,あたらよ,2YQ8TlTmNheRI3VafoDpod,38331,2021-10-04,96,Japan
265,花束,back number,3QIAwtEEDOrv0g5NKCGrXZ,38136,2021-10-04,97,Japan


**Question 1.2** (20 points) Now you need to go to Spotify platform to use its API to further get more information. You could find very detailed [documentation](https://developer.spotify.com/documentation/web-api/) that should guide you with the entire process. 

First, you need to get the audio features from the songs in the ```spotify_top_songs```. You could check the API for getting audio features for several tracks [here](https://developer.spotify.com/console/get-audio-features-several-tracmks/). Essentially, you need to call the [API endpoint](https://developer.spotify.com/console/get-audio-features-several-tracks/), which gives the very detailed explanations. Then you should receive the [Audio feature object](https://developer.spotify.com/documentation/web-api/reference/#object-audiofeaturesobject) in json files, save it as the dataframe ```spotify_top_songs_acoustic_features``` with these features:
- danceability
- energy
- key
- loudness
- mode
- speechiness
- acousticness
- instrumentalness
- liveness
- valence
- tempo
- id
- duration_ms
- time_signature

Note: if you are not able to get this data, download the csv file from the moodle to continue the analysis, but you will not receive the grade from this question.

Hint1: when you request acoustic features from multiple tracks, the url would involve the track id connected by ```%2C```. For example, for two tracks STAY (4JpKVNYnVcJ8tuMKjAj50A), and INDUSTRY Baby (5Z9KJZvQzH6PFmb8SNkxuk), you could search for its url as: `https://api.spotify.com/v1/audio-features?ids=4JpKVNYnVcJ8tuMKjAj50A%2C5Z9KJZvQzH6PFmb8SNkxuk`

Hint2: Spotify requires certain authentication (token) to have access to its data. You need to go to Spotify [developer platform](https://developer.spotify.com/console/get-audio-features-several-tracks/) to request a token and include the token in the requests. It may get expired if you have not used it for a while, then you just need to request a new one.

Hint3: Spotify restricts the number of tracks to be requested in each API call (up to 100), so you may need to do it several times seprately and then combine them later.

In [7]:
# request a new token from Spotify to replace the below one
access_token = 'BQBKL6tNDSbiA8oxnXz-C6iLflBwgGDMv2seDDR33zwP6cEY2MI-epp-z7iaYVsp17ciTwpZtyJCSYOOPuA--W8h_7CAUeEYRmv4j30OmUcHebPkU_qxJT4OK3w-OavrXu24T3-XlO9C'
headers = {
    'Authorization': 'Bearer {token}'.format(token=access_token)
}

In [8]:
song_id = spotify_top_songs["Song ID"].unique().tolist() #find the unique songs in the concatenated dataframe and save the entries as a list
listpt1 = song_id[:100]
listpt2 = song_id[100:200]
listpt3 = song_id[200:] #split the list into 3 parts due to restricted # of api calls

url =  "https://api.spotify.com/v1/audio-features?ids=" #not super efficient, but don't know if there's a better option
url2 =  "https://api.spotify.com/v1/audio-features?ids="
url3 =  "https://api.spotify.com/v1/audio-features?ids=" #define a seperate url for each list

url = url + '%2C'.join(listpt1) #limit the number of api calls and define one big appended url instead of several individual calls
response1 = requests.get(url, headers=headers).json()
listpt1_df = pd.DataFrame(response1['audio_features'], columns=['danceability','energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'id', 'duration_ms', 'time_signature'])

url2 = url2 + '%2C'.join(listpt2)
response2 = requests.get(url2, headers=headers).json()
listpt2_df = pd.DataFrame(response2['audio_features'], columns=['danceability','energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'id', 'duration_ms', 'time_signature'])

url3 = url3 + '%2C'.join(listpt3)
response3 = requests.get(url3, headers=headers).json()
listpt3_df = pd.DataFrame(response3['audio_features'], columns=['danceability','energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'id', 'duration_ms', 'time_signature'])

frames2 = (listpt1_df, listpt2_df, listpt3_df)
spotify_top_songs_acoustic_features = pd.concat(frames2).reset_index(drop=True)
spotify_top_songs_acoustic_features.rename(columns = {'id':'Song ID'}, inplace = True)
spotify_top_songs_acoustic_features

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,Song ID,duration_ms,time_signature
0,0.591,0.764,1,-5.484,1,0.0483,0.03830,0.000000,0.1030,0.478,169.928,5PjdY0CKGZdEuoNab3yDmX,141806,4
1,0.741,0.691,10,-7.395,0,0.0672,0.02210,0.000000,0.0476,0.892,150.087,5Z9KJZvQzH6PFmb8SNkxuk,212353,4
2,0.761,0.525,11,-6.900,1,0.0944,0.44000,0.000007,0.0921,0.531,80.870,02MWAaffLxlfxAUY7c5dvx,238805,4
3,0.588,0.701,9,-6.390,1,0.0402,0.00813,0.000000,0.2000,0.443,104.988,3FeVmId7tL5YN8B7R3imoM,228000,4
4,0.808,0.897,11,-3.712,0,0.0348,0.04690,0.000031,0.3640,0.591,126.026,6PQ88X9TkUIAUIZJHW2upE,231041,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
262,0.825,0.652,1,-3.183,0,0.0802,0.58100,0.000000,0.0931,0.931,95.977,7qiZfU4dY1lWllzX7mPBI3,233713,4
263,0.593,0.663,0,-6.325,0,0.0834,0.06640,0.000000,0.3060,0.559,186.083,118KEprYqS3XNWdBoKzkEH,168982,4
264,0.262,0.470,5,-4.663,1,0.0433,0.04290,0.000000,0.2360,0.307,75.096,2YQ8TlTmNheRI3VafoDpod,332286,4
265,0.519,0.713,2,-3.612,1,0.0324,0.01780,0.000000,0.3530,0.505,159.963,3QIAwtEEDOrv0g5NKCGrXZ,286053,4


**Quesion 1.3** (5 points) 
Merge dataframes ```spotify_top_songs_acoustic_features``` with ```spotify_top_songs``` and to enrich with the acoustic features, check the resulting number of rows and columns.

In [9]:
# Question 1.3
spotify_top_songs_acoustic_features.rename(columns = {'id':'Song ID'}, inplace = True)
spotify_merged_top_acoustic = pd.merge(spotify_top_songs, spotify_top_songs_acoustic_features, on = 'Song ID', how = 'left')
spotify_merged_top_acoustic
#print(spotify_top_song_acoustic.info())


,Title,Artist,Song ID,Streams,Chart Date,Rank,Country,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,STAY (with Justin Bieber),The Kid LAROI,5PjdY0CKGZdEuoNab3yDmX,7714466,2021-09-30,1,Global,0.591,0.764,1,-5.484,1,0.0483,0.03830,0.000000,0.1030,0.478,169.928,141806,4
1,INDUSTRY BABY (feat. Jack Harlow),Lil Nas X,5Z9KJZvQzH6PFmb8SNkxuk,6517968,2021-09-30,2,Global,0.741,0.691,10,-7.395,0,0.0672,0.02210,0.000000,0.0476,0.892,150.087,212353,4
2,Heat Waves,Glass Animals,02MWAaffLxlfxAUY7c5dvx,4460880,2021-09-30,3,Global,0.761,0.525,11,-6.900,1,0.0944,0.44000,0.000007,0.0921,0.531,80.870,238805,4
3,My Universe,"Coldplay, BTS",3FeVmId7tL5YN8B7R3imoM,4142687,2021-09-30,4,Global,0.588,0.701,9,-6.390,1,0.0402,0.00813,0.000000,0.2000,0.443,104.988,228000,4
4,Bad Habits,Ed Sheeran,6PQ88X9TkUIAUIZJHW2upE,4077321,2021-09-30,5,Global,0.808,0.897,11,-3.712,0,0.0348,0.04690,0.000031,0.3640,0.591,126.026,231041,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1495,10月無口な君を忘れる,あたらよ,2YQ8TlTmNheRI3VafoDpod,38331,2021-10-04,96,Japan,0.262,0.470,5,-4.663,1,0.0433,0.04290,0.000000,0.2360,0.307,75.096,332286,4
1496,花束,back number,3QIAwtEEDOrv0g5NKCGrXZ,38136,2021-10-04,97,Japan,0.519,0.713,2,-3.612,1,0.0324,0.01780,0.000000,0.3530,0.505,159.963,286053,4
1497,優しい彗星,YOASOBI,19fhOFi6pNGeZe5uiFlm7c,37380,2021-10-04,98,Japan,0.715,0.722,5,-4.620,1,0.0348,0.19700,0.000000,0.1140,0.522,89.995,215333,4
1498,のびしろ,Creepy Nuts,3bbIIVIwBoLqVcLebiEJFo,37239,2021-10-04,99,Japan,0.669,0.928,11,-2.926,0,0.0699,0.25200,0.000000,0.0686,0.963,174.004,236840,4


In [10]:
spotify_merged_top_acoustic.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1500 entries, 0 to 1499
Data columns (total 20 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Title             1500 non-null   object 
 1   Artist            1500 non-null   object 
 2   Song ID           1500 non-null   object 
 3   Streams           1500 non-null   object 
 4   Chart Date        1500 non-null   object 
 5   Rank              1500 non-null   int64  
 6   Country           1500 non-null   object 
 7   danceability      1500 non-null   float64
 8   energy            1500 non-null   float64
 9   key               1500 non-null   int64  
 10  loudness          1500 non-null   float64
 11  mode              1500 non-null   int64  
 12  speechiness       1500 non-null   float64
 13  acousticness      1500 non-null   float64
 14  instrumentalness  1500 non-null   float64
 15  liveness          1500 non-null   float64
 16  valence           1500 non-null   float64


**Question 1.4** (5 points) Show the top 3 most popular artists in terms of number of unique songs on chart in global, portugal and japan market, respectively.

In [11]:
# Quesion 1.4
#Manual: 
#grouped_glo = spotify_top_songs_global[["Title", "Artist"]].groupby("Artist").nunique().nlargest(3, 'Title')
#grouped_pt = spotify_top_songs_pt[["Title", "Artist"]].groupby("Artist").nunique().nlargest(3, 'Title')
#grouped_jp = spotify_top_songs_jp[["Title", "Artist"]].groupby("Artist").nunique().nlargest(3, 'Title')
#print(grouped_glo, "\n")
#print(grouped_pt, "\n")
#print(grouped_jp)

#Automated
for i in (spotify_top_songs_global,spotify_top_songs_pt, spotify_top_songs_jp):  
    print(i.groupby("Artist").nunique().nlargest(3, 'Title')["Title"], "\n")

Artist
Olivia Rodrigo    7
Doja Cat          5
Billie Eilish     4
Name: Title, dtype: int64 

Artist
Doja Cat          4
Olivia Rodrigo    4
Billie Eilish     3
Name: Title, dtype: int64 

Artist
YOASOBI     13
BTS          7
HIRAIDAI     6
Name: Title, dtype: int64 



**Question 1.5** (5 points) Show average value of acousitc features of songs in global market by the distribution of duration at quartile (0-25%, 25-50%, 50-75%, 75-100%). 

In [31]:
# Question 1.5
quartiles = ['0-25%', '25-50%', '50-75%', '75-100%']
spotify_merged_average_afeatures= spotify_merged_top_acoustic[spotify_merged_top_acoustic["Country"]=="Global"].drop_duplicates(subset=['Song ID']).reset_index(drop=True)

spotify_merged_average_afeatures["quartile"]= pd.qcut(spotify_merged_average_afeatures['duration_ms'], q=4, labels=quartiles)

pivot_1 = spotify_merged_average_afeatures.pivot_table(index = ["quartile"], aggfunc = "mean")
pivot_1.drop('Rank', inplace=True, axis=1)
pivot_1

,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
quartile,,,,,,,,,,,,,
0-25%,0.236579,0.687724,149746.172414,0.669379,0.022901,5.827586,0.170731,-5.686793,0.482759,0.101397,125.483103,3.965517,0.524966
25-50%,0.244524,0.706552,178988.758621,0.674103,0.003462,4.862069,0.179862,-5.192310,0.689655,0.091662,128.085862,3.896552,0.629069
50-75%,0.253050,0.693821,207381.214286,0.688429,0.000066,4.928571,0.163929,-5.571000,0.607143,0.076025,124.055429,4.000000,0.553393
75-100%,0.225980,0.661931,254341.551724,0.603000,0.004259,6.103448,0.134655,-6.181621,0.689655,0.101766,124.879241,3.862069,0.429514


**Question 1.6** (5 points) Show the top 3 artists with the most total streams in global, portugal and japan markets.

In [42]:
# Question 1.6
#Manual version:
'''
spotify_top_songs_global['Streams'] = spotify_top_songs_global['Streams'].astype(int)
spotify_top_songs_pt['Streams'] = spotify_top_songs_pt['Streams'].astype(int)
spotify_top_songs_jp['Streams'] = spotify_top_songs_jp['Streams'].astype(int)
grouped_glosum = spotify_top_songs_global[["Artist", "Streams"]].groupby("Artist").sum().sort_values(ascending = False, by ="Streams").head(3)
grouped_ptsum = spotify_top_songs_pt[["Artist", "Streams"]].groupby("Artist").sum().sort_values(ascending = False, by ="Streams").head(3)
grouped_jpsum = spotify_top_songs_jp[["Artist", "Streams"]].groupby("Artist").sum().sort_values(ascending = False, by ="Streams").head(3)
print(grouped_glosum, "\n")
print(grouped_ptsum, "\n")
print(grouped_jpsum)
'''

#Automated:
for i in (spotify_top_songs_global, spotify_top_songs_pt, spotify_top_songs_jp): 
        i['Streams'] = i['Streams'].astype(int)
        print(i['Country'].unique()[0])
        print(i.groupby("Artist").sum().sort_values(ascending = False, by ="Streams").head(3)["Streams"], "\n")

Global
Artist
Lil Nas X         64552221
Doja Cat          58792737
Olivia Rodrigo    55254893
Name: Streams, dtype: int64 

Portugal
Artist
Lil Nas X    490634
CKay         320281
Doja Cat     314178
Name: Streams, dtype: int64 

Japan
Artist
YOASOBI                  7197817
BTS                      4291855
Official HIGE DANdism    3138171
Name: Streams, dtype: int64 



**Question 1.7** (5 points) Show the number of songs across the keys (row) and (Portugal/Japan) market (column).

In [41]:
# Question 1.7
jp_data = spotify_merged_top_acoustic[spotify_merged_top_acoustic["Country"]=="Japan"][['Song ID', 'Country', 'key']].drop_duplicates(subset=('Song ID')).reset_index(drop=True)
jp_keys = jp_data.groupby('key').count()
pt_data = spotify_merged_top_acoustic[spotify_merged_top_acoustic["Country"]=="Portugal"][['Song ID', 'Country', 'key']].drop_duplicates(subset=('Song ID')).reset_index(drop=True)
pt_keys = pt_data.groupby('key').count()

data_merge = pd.merge(jp_keys, pt_keys, on = "key", how = 'right')
data_merge.rename(columns = {'Song ID_x':'Keys Japan'},inplace = True)
data_merge.rename(columns = {'Song ID_y':'Keys Portugal'},inplace = True)
data_merge[["Keys Japan", "Keys Portugal"]]

,Keys Japan,Keys Portugal
key,,
0,5,11
1,16,16
2,11,6
3,4,5
4,3,4
5,10,12
6,6,9
7,10,11
8,12,17


**Question 1.8** (5 points) Show the top 5 artists that has the most number of songs-days in global market (if a song appeared in 2 days, it will be counted as the 2 song-days.

In [15]:
# Question 1.8
artist_days_glo = spotify_top_songs_global[["Artist", "Song ID"]].groupby("Artist").count().nlargest(5, 'Song ID')
artist_days_glo

,Song ID
Artist,
Olivia Rodrigo,32
Doja Cat,25
Billie Eilish,20
Drake,20
The Weeknd,20


**Question 1.9** (10 points) Compare the acoustic features of top songs in Portugal and in Japan, by checking the correlations between rank and acoustic features using Pearman (it's Pearson, isn't it?) and Spearman correlations.


In [16]:
# What do we have to do here? -> Understand, which effect each acoustic feature has on the rank of a song
jp_afeatures = spotify_merged_top_acoustic[spotify_merged_top_acoustic["Country"]=="Japan"].reset_index(drop = True) #select only the entries for the particular nation
pt_afeatures = spotify_merged_top_acoustic[spotify_merged_top_acoustic["Country"]=="Portugal"].reset_index(drop = True)

corr_pears_pt = pt_afeatures[pt_afeatures.columns].corr(method = "pearson")["Rank"] #calculate the correlation focussing on the rank
corr_spear_pt = pt_afeatures[pt_afeatures.columns].corr(method = "spearman")["Rank"]

corr_pears_jp = jp_afeatures[jp_afeatures.columns].corr(method = "pearson")["Rank"]
corr_spear_jp = jp_afeatures[jp_afeatures.columns].corr(method = "spearman")["Rank"]

pt_merge = pd.concat([corr_pears_pt, corr_spear_pt], axis = 1, keys=['pearson_pt', 'spearman_pt'])
jp_merge = pd.concat([corr_pears_jp, corr_spear_jp], axis = 1, keys=['pearson_jp', 'spearman_jp']) 

all_merge = pd.concat([pt_merge, jp_merge], axis = 1, keys=['Portugal', 'Japan']).drop("Rank", axis = 0) #create one large dataframe of the 4 columns
all_merge

Portugal                  Japan            
                 pearson_pt spearman_pt pearson_jp spearman_jp
danceability       0.083782    0.083223  -0.022101   -0.011061
energy            -0.017800   -0.018528  -0.014775   -0.011571
key               -0.059467   -0.068003  -0.056961   -0.051502
loudness          -0.166626   -0.144227   0.053387    0.060453
mode              -0.015522   -0.015522  -0.028735   -0.028735
speechiness        0.024322   -0.032392   0.051946    0.122617
acousticness       0.021286    0.004246  -0.001515    0.032590
instrumentalness  -0.065531   -0.110757   0.115483    0.159398
liveness          -0.087367    0.047954  -0.183397   -0.177359
valence           -0.086033   -0.084194  -0.004570   -0.006005
tempo              0.023731    0.009893  -0.021304   -0.027492
duration_ms        0.045797    0.046917   0.102141    0.095879
time_signature     0.025701    0.024947        NaN         NaN

**Question 1.10** (10 points) 
Compare the acoustic features of top songs in Portugal and in Japan, by checking whether the differences between feature values are statistically significant or not. Show the features ranked by the absolute magnitude of differences with statistical significance level of at least p<0.05.

In [29]:
import scipy
from scipy import stats
from scipy.stats import ttest_ind, ttest_ind_from_stats
features = [all_merge.index]
t_output = []

for i in features:
    t_output.append(ttest_ind(pt_afeatures[i], jp_afeatures[i], equal_var=False)[1])
    t_output.append(abs(ttest_ind(pt_afeatures[i], jp_afeatures[i], equal_var=False)[0]))

test_df = pd.DataFrame(t_output, columns = all_merge.index)
t_step = test_df.T
t_step.rename(columns={0:"p-value"}, inplace = True)
t_step.rename(columns={1:"absolute magnitdude"}, inplace = True)
t_result = t_step.sort_values(["p-value"], axis=0, ascending=True)
t_result

,p-value,absolute magnitdude
duration_ms,1.128534e-37,13.380514
loudness,9.845541e-32,12.186581
energy,1.720614e-29,11.648145
acousticness,1.660111e-27,11.259583
speechiness,9.565124e-27,11.165100
danceability,2.803773e-23,10.194326
mode,8.632713e-19,9.035951
liveness,1.721071e-12,7.149613
instrumentalness,5.279709e-04,3.488753
valence,1.892279e-03,3.115049
